In [1]:
from __future__ import print_function

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

import pickle

import numpy as np

import pandas as pd

#% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set()

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

from sklearn.preprocessing import StandardScaler

import joblib

#from libs.container import Container
#from libs.display import d
#from libs.experiment import KFoldExperiment, WithAnotherExperiment, roc

def my_distance(x, y, dtype=np.float32):
    import math
    d = math.dist(x, y)
    return np.float32(d)

In [2]:
#cpu = joblib.cpu_count()

In [3]:
import galaxychop as gchop

gal = gchop.preproc.center_and_align(gchop.io.read_hdf5("tests/datasets/galaxy_TNG_611399.h5"))
circ = gchop.preproc.jcirc(gal)
df = pd.DataFrame({
    "eps": circ.eps,
    "eps_r": circ.eps_r,
    "normalized_star_energy": circ.normalized_star_energy}).dropna()
X = df.to_numpy()

In [1]:
#df = pd.concat([data.b278, data.b261])

#cls = {name: idx for idx, name in enumerate(df.tile.unique())}
#df["cls"] = df.tile.apply(cls.get)

#print(cls)

df, hue = gal.plot.get_df_and_hue(ptypes=None, attributes=["x", "y", "z"], labels="ptype", lmap=None)

df = df[df["ptype"] == "stars"].drop(columns=['ptype'])

from scipy.spatial.distance import squareform, pdist

distance_matrix = pd.DataFrame(squareform(pdist(df.iloc[:, 1:])))

NameError: name 'gal' is not defined

In [ ]:
distance_array = distance_matrix.values.flatten()

fst_percentile = np.percentile(distance_array, 25)

distance_threshold = fst_percentile

: 

: 

In [ ]:
[item for sublist in X[:, (0,)] for item in sublist]
#np.linspace(1e-3, 1, 100)

#print(X[:, (1,)])

# calcular todas las distancias y buscaria el primer cuartil


# buscar mejores valores por cantidad de cluster
# graficar el que dio los mejores resultados.

# correr una vez con compute_full_tree para ver que onda


[1.0,
 -0.46203332371464234,
 0.715853709458445,
 -0.8424471985968337,
 -0.12937387517287627,
 -0.5553402096431244,
 0.528708672702196,
 -0.40451617138659024,
 0.49674051971044036,
 0.15439942479934532,
 0.10547570080717363,
 -0.38452007204231986,
 0.33260201732870875,
 0.007612508336009978,
 -0.37622746213532116,
 0.18373878854248848,
 -0.1548503532168495,
 -0.695747059453609,
 0.20658123836006262,
 -0.09466221038321161,
 0.08179162093740514,
 0.03188701008695509,
 -0.030026170910725136,
 0.30093944787353866,
 -0.9071090364104016,
 0.47764994727220245,
 0.8396567436702752,
 -0.15054766332736108,
 -0.05608281264142023,
 -0.2934092707106602,
 0.15455827865702465,
 0.49678789216005803,
 -0.15429907125119208,
 -0.6560047885780937,
 0.9488288390430003,
 -0.6468306266080502,
 0.9371478162124188,
 0.6869384565247173,
 -0.6192076409465158,
 -0.1403698380486232,
 -0.6791837972998965,
 0.6574444172033179,
 0.9859914116146895,
 -0.16936325740881022,
 0.06909361148203348,
 0.3063102360335382,
 0.

In [ ]:
#X = df[X_columns].values
#y = df.cls.values
class WrappedAgglomerativeClustering(AgglomerativeClustering):
    def __init__(
        self,
        n_clusters=2,
        *,
        affinity="euclidean",
        memory=None,
        connectivity=None,
        compute_full_tree="auto",
        linkage="ward",
        distance_threshold=None,
        compute_distances=False,
        columns=[],
    ):
        self.columns = columns
        super().__init__(n_clusters=n_clusters, distance_threshold=distance_threshold, memory=memory, connectivity=connectivity, compute_full_tree=compute_full_tree, linkage=linkage, affinity=affinity, compute_distances=compute_distances)

    def fit(self, X, y=None):
        X_with_selected_columns = X[:, self.columns]
        super().fit(X=X_with_selected_columns, y=y)

In [6]:
#%%time
# Split the dataset in two equal parts
#X_train, X_test, y_train, y_test = train_test_split(
#    X, y, test_size=0.2, random_state=0)

# Set the parameters by AgglomerativeClustering (hierarchical clustering)
import itertools

def combinations_up_to(elements, n):
    all_combinations = []
    _n = min(n+1, len(elements) + 1)
    for i in range(1, _n):
        all_combinations.append(list(itertools.combinations(elements, i)))

    flat_list = [item for sublist in all_combinations for item in sublist]
    return flat_list

tuned_parameters = [
    {'n_clusters': [2, 3, 4, 5], 'linkage' : ['ward', 'complete', 'average', 'single'], 'columns' : combinations_up_to([0, 1, 2], 3)},
]
# Deberia usar el parametro distance_threshold?
# Deberia variar la metrica usada? Por defecto es la eucladiana y no se si tiene sentido probar con otras.

#sklearn.metrics.silhouette_score
#sklearn.metrics.davies_bouldin_score


model = WrappedAgglomerativeClustering()
#Hacer nuevo cluster que sea como wrapper, que filtre por columnas los datos que se le pasan y despues pasa kwarts al clustering


def my_silhouette_score(model, X, y=None):
    preds = model.fit_predict(X)
    return silhouette_score(X, preds) if len(set(preds)) > 1 else float('nan')

def cv_silhouette_scorer(estimator, X):
    estimator.fit(X)
    cluster_labels = estimator.labels_
    num_labels = len(set(cluster_labels))
    num_samples = len(X.index)
    if num_labels == 1 or num_labels == num_samples:
        return -1
    else:
        return sc(X, cluster_labels)

# We have to call the function with "_loss" so that GridSearachCV use the function as a minimizer,
# since david_bouldin score is better when closer to 0.
#scorer_f
def davies_bouldin_loss(model, X, y=None):
    preds = model.fit_predict(X)
    return davies_bouldin_score(X, preds)

from sklearn.metrics import make_scorer

#grid_search_silhouette_score = make_scorer(my_silhouette_score, greater_is_better=True)
#grid_search_david_bouldin_loss = make_scorer(my_david_bouldin_loss, greater_is_better=False)

scores = [davies_bouldin_loss, my_silhouette_score]
cpu = 1
# To avoid doing cross validation
cv=[(slice(None), slice(None))]

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(model, tuned_parameters, n_jobs=cpu,
                       scoring=score)
    clf.fit(X)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    #y_true, y_pred = y_test, clf.predict(X_test)
    #print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for <function davies_bouldin_loss at 0x7f2d62b8d000>

Best parameters set found on development set:

{'columns': (2,), 'linkage': 'ward', 'n_clusters': 5}

Grid scores on development set:

0.785 (+/-0.121) for {'columns': (0,), 'linkage': 'ward', 'n_clusters': 2}
0.988 (+/-0.124) for {'columns': (0,), 'linkage': 'ward', 'n_clusters': 3}
1.122 (+/-0.157) for {'columns': (0,), 'linkage': 'ward', 'n_clusters': 4}
1.358 (+/-0.189) for {'columns': (0,), 'linkage': 'ward', 'n_clusters': 5}
0.617 (+/-0.229) for {'columns': (0,), 'linkage': 'complete', 'n_clusters': 2}
0.871 (+/-0.073) for {'columns': (0,), 'linkage': 'complete', 'n_clusters': 3}
0.960 (+/-0.181) for {'columns': (0,), 'linkage': 'complete', 'n_clusters': 4}
1.161 (+/-0.176) for {'columns': (0,), 'linkage': 'complete', 'n_clusters': 5}
0.509 (+/-0.390) for {'columns': (0,), 'linkage': 'average', 'n_clusters': 2}
0.741 (+/-0.181) for {'columns': (0,), 'linkage': 'average', 'n_clusters': 3}
0.838 (+/-0.2

In [22]:
list(range(5, 15)) + [np.sqrt(len(X_columns))]

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_

In [20]:
%%time
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [
    {'max_features': ['auto', 'sqrt', "log2", None, 0.2, 0.5], 
     "min_samples_split": [2, 5, 10],
     "n_estimators": [500], 
     "criterion": ["entropy"], 
     "n_jobs": [10]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5, n_jobs=cpu,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

NameError: name 'y' is not defined

In [18]:
# Random Forest 
RF_prec = {'max_features': None, 'min_samples_split': 10, 'n_jobs': 10, 'criterion': 'entropy', 'n_estimators': 500}
RF_recall = {'max_features': None, 'min_samples_split': 10, 'n_jobs': 10, 'criterion': 'entropy', 'n_estimators': 500}

RF_prec == RF_recall

True

In [19]:
SVM_prec = {'kernel': 'rbf', 'C': 1000, 'gamma': 0.001}
SVM_recall = {'kernel': 'rbf', 'C': 1000, 'gamma': 0.001}

SVM_recall == SVM_prec

True